<h1 align="center">Предобработка данных</h1>

<h2>📚 Импорт библиотек</h2>

In [1]:
import os
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="sklearn")

from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split

<h2>📁 Загрузка данных</h2>

In [2]:
current_dir = os.getcwd()
print(f"Текущая папка: {current_dir}")

project_root = os.path.dirname(os.getcwd())
data_path = os.path.join(project_root, 'data', 'raw', 'car_data.csv')
print(f"Путь к данным: {data_path}")

df = pd.read_csv(data_path)
print(f"✅ Данные загружены: {df.shape[0]} строк, {df.shape[1]} столбцов")

Текущая папка: C:\Users\airat.tagirov\Desktop\CarPricePrediction\notebooks
Путь к данным: C:\Users\airat.tagirov\Desktop\CarPricePrediction\data\raw\car_data.csv
✅ Данные загружены: 205 строк, 26 столбцов


<h2>🔄 Предобработка CarName</h2>
<p>Повторяем обработку из EDA для извлечения бренда</p>

In [3]:
# Повторяем обработку CarName из EDA
brand_correction = {
    'maxda': 'mazda', 'porcshce': 'porsche', 'toyouta': 'toyota',
    'vokswagen': 'volkswagen', 'vw': 'volkswagen'
}
df['CarName'] = df['CarName'].replace(brand_correction, regex=True)
df['CarName'] = df['CarName'].str.lower()
df['brand'] = df['CarName'].str.split().str[0]

df.shape

(205, 27)

<h2>1. Удаление ненужных столбцов</h2>
<p>Удаляем признаки с низкой корреляцией и технические столбцы</p>


In [4]:
for_delete = ['carheight', 'stroke', 'compressionratio', 'peakrpm', 'car_ID', 'CarName']

df.drop(for_delete, axis=1, inplace=True)

In [5]:
print(f"Осталось столбцов: {df.shape[1]}")
print(df.columns)

Осталось столбцов: 21
Index(['symboling', 'fueltype', 'aspiration', 'doornumber', 'carbody',
       'drivewheel', 'enginelocation', 'wheelbase', 'carlength', 'carwidth',
       'curbweight', 'enginetype', 'cylindernumber', 'enginesize',
       'fuelsystem', 'boreratio', 'horsepower', 'citympg', 'highwaympg',
       'price', 'brand'],
      dtype='object')


<h2>2. Обработка редких категорий</h2>
<p>Объединяем бренды с малым количеством наблюдений</p>

In [6]:
other = df['brand'].value_counts() < 5
rare_brands = other[other == True].index
df['brand'] = df['brand'].replace(rare_brands, 'other')

In [7]:
print(df['brand'].value_counts())

brand
toyota        32
nissan        18
mazda         17
other         16
honda         13
mitsubishi    13
subaru        12
volkswagen    12
peugeot       11
volvo         11
dodge          9
bmw            8
buick          8
audi           7
plymouth       7
saab           6
porsche        5
Name: count, dtype: int64


<h2>3. Создание новых признаков</h2>
<p>Фич инжиниринг: соотношения параметров</p>

In [8]:
df['power_to_weight'] = df['horsepower'] / df['curbweight']
df['mpg_avg'] = (df['citympg'] + df['highwaympg']) / 2
df['size_ratio'] = df['carwidth'] / df['carlength']

df.head()

,symboling,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,carlength,carwidth,...,fuelsystem,boreratio,horsepower,citympg,highwaympg,price,brand,power_to_weight,mpg_avg,size_ratio
0,3,gas,std,two,convertible,rwd,front,88.6,168.8,64.1,...,mpfi,3.47,111,21,27,13495.0,other,0.043564,24.0,0.379739
1,3,gas,std,two,convertible,rwd,front,88.6,168.8,64.1,...,mpfi,3.47,111,21,27,16500.0,other,0.043564,24.0,0.379739
2,1,gas,std,two,hatchback,rwd,front,94.5,171.2,65.5,...,mpfi,2.68,154,19,26,16500.0,other,0.054552,22.5,0.382593
3,2,gas,std,four,sedan,fwd,front,99.8,176.6,66.2,...,mpfi,3.19,102,24,30,13950.0,audi,0.043646,27.0,0.374858
4,2,gas,std,four,sedan,4wd,front,99.4,176.6,66.4,...,mpfi,3.19,115,18,22,17450.0,audi,0.040722,20.0,0.375991


<h3>📊 Корреляция новых признаков с ценой</h3>


In [9]:
numeric_cols = ['mpg_avg','power_to_weight','size_ratio', 'price']

price_correlations = df[numeric_cols].corr()['price']
print("📈 Признаки по корреляции с ценой:")
print(price_correlations)

📈 Признаки по корреляции с ценой:
mpg_avg           -0.696835
power_to_weight    0.533767
size_ratio        -0.486014
price              1.000000
Name: price, dtype: float64


<p>mpg_avg: -0.70 (сильная обратная связь - экономичные авто дешевле)</p>
<p>power_to_weight: 0.53 (средняя прямая связь - мощные авто дороже)</p>
<p>size_ratio: -0.49 (средняя обратная связь)</p>

<h2>4. Разделение на train/test</h2>
<p><strong>Почему ДО масштабирования и кодирования:</strong></p>
<ul>
  <li>Предотвращение data leakage - модель не должна "видеть" тестовые данные</li>
  <li>Кодировщики и скейлеры должны обучаться ТОЛЬКО на тренировочных данных</li>
  <li>Тестовые данные преобразуются теми же параметрами, но без переобучения</li>
  <li>Правильный подход для воспроизводимости в продакшене</li>
</ul>

In [10]:
X = df.drop('price', axis=1)
y = df['price']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=42,
    stratify=df['brand']
)

comparison = pd.concat([
    X_train['brand'].value_counts(normalize=True).round(3),
    X_test['brand'].value_counts(normalize=True).round(3)],
    axis=1, keys=['Train', 'Test'])
print("Сравнение распределения брендов:")
print(comparison)

Сравнение распределения брендов:
            Train   Test
brand                   
toyota      0.159  0.146
nissan      0.085  0.098
mazda       0.085  0.073
other       0.079  0.073
honda       0.061  0.073
volkswagen  0.061  0.049
mitsubishi  0.061  0.073
subaru      0.061  0.049
volvo       0.055  0.049
peugeot     0.055  0.049
dodge       0.043  0.049
buick       0.037  0.049
bmw         0.037  0.049
plymouth    0.037  0.024
saab        0.030  0.024
audi        0.030  0.049
porsche     0.024  0.024


<h2>5. Кодирование категориальных признаков</h2>
<p>One-Hot Encoding и Label Encoding в зависимости от типа признака</p>

In [11]:
distribution = df.select_dtypes(include=[object]).nunique() < 4

for_label = distribution[distribution == True].index.tolist()
print(f"Label Encoding: {for_label}")
for_onehot = distribution[distribution == False].index.tolist()
print(f"One-Hot Encoding: {for_onehot}")

le = LabelEncoder()

for column in for_label:
    X_train[column] = le.fit_transform(X_train[column])
    X_test[column] = le.transform(X_test[column])

ohe = OneHotEncoder(sparse_output=False, drop='first')

for column in for_onehot:
    ohe = OneHotEncoder(sparse_output=False, drop='first', handle_unknown='ignore')  # НОВЫЙ encoder для каждого столбца
    
    # X_train
    onehot_encoded = ohe.fit_transform(X_train[[column]])
    feature_names = ohe.get_feature_names_out([column])
    onehot_df = pd.DataFrame(onehot_encoded, columns=feature_names, index=X_train.index)
    
    X_train = pd.concat([X_train, onehot_df], axis=1)
    X_train.drop(column, axis=1, inplace=True)
    
    # X_test - используем ТОТ ЖЕ ohe
    onehot_encoded = ohe.transform(X_test[[column]])
    onehot_df = pd.DataFrame(onehot_encoded, columns=feature_names, index=X_test.index)
    
    X_test = pd.concat([X_test, onehot_df], axis=1)
    X_test.drop(column, axis=1, inplace=True)

Label Encoding: ['fueltype', 'aspiration', 'doornumber', 'drivewheel', 'enginelocation']
One-Hot Encoding: ['carbody', 'enginetype', 'cylindernumber', 'fuelsystem', 'brand']


In [12]:
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"Колонки X_train: {X_train.columns.tolist()}")

X_train shape: (164, 55)
X_test shape: (41, 55)
Колонки X_train: ['symboling', 'fueltype', 'aspiration', 'doornumber', 'drivewheel', 'enginelocation', 'wheelbase', 'carlength', 'carwidth', 'curbweight', 'enginesize', 'boreratio', 'horsepower', 'citympg', 'highwaympg', 'power_to_weight', 'mpg_avg', 'size_ratio', 'carbody_hardtop', 'carbody_hatchback', 'carbody_sedan', 'carbody_wagon', 'enginetype_l', 'enginetype_ohc', 'enginetype_ohcf', 'enginetype_ohcv', 'enginetype_rotor', 'cylindernumber_five', 'cylindernumber_four', 'cylindernumber_six', 'cylindernumber_three', 'cylindernumber_twelve', 'cylindernumber_two', 'fuelsystem_2bbl', 'fuelsystem_4bbl', 'fuelsystem_idi', 'fuelsystem_mpfi', 'fuelsystem_spdi', 'fuelsystem_spfi', 'brand_bmw', 'brand_buick', 'brand_dodge', 'brand_honda', 'brand_mazda', 'brand_mitsubishi', 'brand_nissan', 'brand_other', 'brand_peugeot', 'brand_plymouth', 'brand_porsche', 'brand_saab', 'brand_subaru', 'brand_toyota', 'brand_volkswagen', 'brand_volvo']


In [13]:
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print("Пропуски в X_train:", X_train.isnull().sum().sum())
print("Пропуски в X_test:", X_test.isnull().sum().sum())

X_train shape: (164, 55)
X_test shape: (41, 55)
Пропуски в X_train: 0
Пропуски в X_test: 0


<h2>6. Масштабирование числовых признаков</h2>
<p>StandardScaler для нормализации числовых данных</p>

In [14]:
scaler = StandardScaler()
orig_num_col = df.select_dtypes(include=[np.number]).columns.drop('price')
X_train[orig_num_col] = scaler.fit_transform(X_train[orig_num_col])
X_test[orig_num_col] = scaler.transform(X_test[orig_num_col])

result = pd.concat([
    X_train[orig_num_col].describe().round(2), 
    X_test[orig_num_col].describe().round(2)
], axis=1, keys=['Train', 'Test'])

print("После масштабирования:")
print(result.T)

После масштабирования:
                       count  mean   std   min   25%   50%   75%   max
Train symboling        164.0 -0.00  1.00 -2.25 -0.67  0.12  0.91  1.70
      wheelbase        164.0  0.00  1.00 -2.03 -0.70 -0.28  0.63  3.74
      carlength        164.0  0.00  1.00 -2.71 -0.54 -0.10  0.78  2.81
      carwidth         164.0 -0.00  1.00 -2.66 -0.85 -0.17  0.31  2.95
      curbweight       164.0  0.00  1.00 -2.09 -0.71 -0.28  0.76  2.98
      enginesize       164.0  0.00  1.00 -1.56 -0.69 -0.28  0.36  4.74
      boreratio        164.0  0.00  1.00 -2.48 -0.71 -0.06  0.96  2.28
      horsepower       164.0  0.00  1.00 -1.46 -0.89 -0.24  0.31  4.11
      citympg          164.0  0.00  1.00 -1.91 -0.97 -0.10  0.76  3.74
      highwaympg       164.0 -0.00  1.00 -2.16 -0.84 -0.10  0.49  3.44
      power_to_weight  164.0 -0.00  1.00 -2.15 -0.61 -0.22  0.45  3.72
      mpg_avg          164.0 -0.00  1.00 -1.98 -0.83 -0.14  0.63  3.61
      size_ratio       164.0  0.00  1.00 -1.98 -0.65 -